In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [2]:
import xarray as xr
import numpy as np
import pandas as pd

import ecephys_analyses as ea 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [3]:
xr.set_options(keep_attrs=True)

In [13]:
def summarize(subject, condition, spws, norm_spws):
    return dict(subject=subject, condition=condition, amplitude=spws.sink_amplitude.median() / norm_spws.sink_amplitude.median(), integral=spws.sink_integral.median() / norm_spws.sink_integral.median(), duration=spws.duration.median() / norm_spws.duration.median())

In [14]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
subject_dfs = list()
for subject in subjects:
    norm_hyp, norm_spws = ea.load_baseline_light_period_nrem(subject)
    dep_hyp, dep_spws = ea.load_deprivation(subject)
    early_hyp, early_spws = ea.load_early_recovery_nrem(subject)
    late_hyp, late_spws = ea.load_late_recovery_nrem(subject)
    rec_2h_hyp, rec_2h_spws = ea.load_first2h_recovery_nrem(subject)
    rec_6h_hyp, rec_6h_spws = ea.load_recovery_nrem(subject)
    bsl_2h_hyp, bsl_2h_spws = ea.load_first2h_recovery_match_nrem(subject, rec_2h_hyp)

    subject_df = pd.DataFrame.from_dict(
        [
            summarize(subject, "baseline-light-period-nrem", norm_spws, norm_spws),
            summarize(subject, "recovery-first2h-match", bsl_2h_spws, norm_spws),
            summarize(subject, "recovery-first2h", rec_2h_spws, norm_spws),
            summarize(subject, "recovery-first1h", early_spws, norm_spws),
            summarize(subject, "recovery-last1h", late_spws, norm_spws),
            summarize(subject, "recovery-full", rec_6h_spws, norm_spws),
            summarize(subject, "deprivation", dep_spws, norm_spws)
        ]
    )

    subject_dfs.append(subject_df)

In [15]:
df = pd.concat(subject_dfs)
df

,subject,condition,amplitude,integral,duration
0,Segundo,baseline-light-period-nrem,1.000000,1.000000,1.000000
1,Segundo,recovery-first2h-match,1.004933,1.009572,1.006667
2,Segundo,recovery-first2h,1.052521,1.076271,1.013333
3,Segundo,recovery-first1h,1.059138,1.087895,0.993333
4,Segundo,recovery-last1h,1.033982,1.042989,1.013333
5,Segundo,recovery-full,1.047568,1.070172,1.013333
6,Segundo,deprivation,1.045213,0.884018,0.833333
0,Valentino,baseline-light-period-nrem,1.000000,1.000000,1.000000
1,Valentino,recovery-first2h-match,1.015197,1.007655,0.992908
2,Valentino,recovery-first2h,1.037010,1.126205,1.085106


In [16]:
df.to_csv('median-spw-properties.csv', index=False)